Task 1


In [4]:
from sympy import symbols, diff, lambdify, Abs

x = symbols('x')
f = x**4

f_prime = diff(f, x)
f_third = diff(f_prime, x, x)

f_lambdified = lambdify(x, f)
f_prime_lambdified = lambdify(x, f_prime)
f_third_lambdified = lambdify(x, f_third)

a, b = 0, 1
x1 = 0.5
h = 0.1

f_2 = f_lambdified(x1 + h)
f_0 = f_lambdified(x1 - h)
actual_f_prime = f_prime_lambdified(x1)

central_diff = (f_2 - f_0) / (2 * h)

M_3 = max(abs(f_third_lambdified(a)), abs(f_third_lambdified(b)))

error_bound = (M_3 * h**2) / 6

actual_error = Abs(actual_f_prime - central_diff)

results = {
    'f_prime(x1)': actual_f_prime,
    'Central Difference Approximation': central_diff,
    'Actual Error': actual_error,
    'Error Bound': error_bound
}
results


{'f_prime(x1)': 0.5,
 'Central Difference Approximation': 0.5199999999999999,
 'Actual Error': 0.0199999999999999,
 'Error Bound': 0.04000000000000001}

Task 2

In [ ]:
import numpy as np
from scipy.optimize import least_squares
import matplotlib.pyplot as plt

def chebyshev_polys(x):
    return [1, x, 2*x**2 - 1, 4*x**3 - 3*x, 8*x**4 - 8*x**2 + 1, 16*x**5 - 20*x**3 + 5*x, 32*x**6 - 48*x**4 + 18*x**2 - 1, 64*x**7 - 112*x**5 + 56*x**3 - 7*x,128*x**8 - 256*x**6 + 160*x**4 - 32*x**2 + 1, 256*x**9 - 576*x**7 + 432*x**5 - 120*x**3 + 9*x]

def normalized_chebyshev_polys(x):
    T = chebyshev_polys(x)
    normalized_T = []
    for poly in T:
        max_coeff = np.max(np.abs(poly))
        normalized_poly = poly / max_coeff if max_coeff != 0 else poly
        normalized_T.append(normalized_poly)
    return normalized_T

def chebyshev_fit(coeffs, x):
    T = normalized_chebyshev_polys(x)
    return sum(c * Tn for c, Tn in zip(coefficients, T))

def residual(coeffs, x, y):
    T = normalized_chebyshev_polys(x)
    approx = sum(c * Tn for c, Tn in zip(coeffs, T))
    return approx - y


coeffs = [1, -2, 3, -4, 5]
#coeffs = [12, 0, 0, 1, 2, 8, -17]
#coeffs = [1, 0, 0, 1]
coeffs = [1,1,1,1,1,1,1,1,1,1,1]
n = len(coeffs) - 1
interval = [0, 1]

target_function = Polynomial(coeffs)
x_vals = np.linspace(interval[0], interval[1], 1000)
y_vals = target_function(x_vals)
initial_coeffs = [0]*n
coefficients = least_squares(residual, initial_coeffs, args=(x_vals, y_vals)).x



fit_vals = chebyshev_fit(coefficients, x_vals)
max_residual = np.max(np.abs(y_vals - fit_vals))


plt.figure(figsize=(10, 6))
plt.plot(x_vals, y_vals, label='Original $f(x)$', color='green')
plt.plot(x_vals, fit_vals, label='Approximating Polynomial $P(x)$', linestyle='dotted', color='red')
plt.title("Comparison")
plt.xlabel("x")
plt.ylabel("Value")
plt.legend()
plt.grid(True)
plt.show()

print("Original Coefficients:",coeffs , "\nMax Residual: ", max_residual, "\nApproximating Polynomial Coefficients:", coefficients)


Task 3

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad

def f(x):
    return x**(1/3)

phi_0 = lambda x: 1
phi_1 = lambda x: x

phi_0_phi_0 = quad(lambda x: phi_0(x)**2, 0, 1)[0]
phi_0_phi_1 = quad(lambda x: phi_0(x)*phi_1(x), 0, 1)[0]
phi_1_phi_1 = quad(lambda x: phi_1(x)**2, 0, 1)[0]

f_phi_0 = quad(lambda x: f(x)*phi_0(x), 0, 1)[0]
f_phi_1 = quad(lambda x: f(x)*phi_1(x), 0, 1)[0]

A = np.array([[phi_0_phi_0, phi_0_phi_1], [phi_0_phi_1, phi_1_phi_1]])
b = np.array([f_phi_0, f_phi_1])
c = np.linalg.solve(A, b)

P = lambda x: c[0]*phi_0(x) + c[1]*phi_1(x)


mse = quad(lambda x: (f(x) - P(x))**2, 0, 1)[0]

x_values = np.linspace(0, 1, 10000)
plt.plot(x_values, f(x_values), label='f(x) = $x^{1/3}$', color = "green")
plt.plot(x_values, P(x_values), label='Best Approximation Polynomial', linestyle='dotted', color = "red")
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.title('Best Mean Squared Approximation')
plt.show()

print("Coeffs:", list(c), "Mean Squared Error:", mse)
